#Synthea Analysis

In [ ]:
from google.colab import drive
drive.mount('/content/drive/', force_remount=True)

Mounted at /content/drive/


In [ ]:
!ls /content/drive/MyDrive/synthea_dataset

'Adrian111_Bartell116_fd25b51b-dc2a-4edb-b7c9-2c5d178a0ee7 (1).json'
 Adrian111_Bartell116_fd25b51b-dc2a-4edb-b7c9-2c5d178a0ee7.json
 Adrianna470_Boyer713_6fcf56ed-8ad8-4395-a966-9ebee3822656.json
 Adrienne302_Zulauf375_1c83fb03-d139-4626-bb39-4a062c22b533.json
 Alaine226_Kassulke119_5efa5654-8190-4f4d-9ba3-3b79448440b5.json
 Ali918_Larson43_896d3035-d036-4c8a-9901-367dc8814495.json
 Allegra202_Heller342_cc4c634f-c2ff-4963-b75a-fa2e33dc208f.json
 Allena79_Pacocha935_d11e7fb0-5853-4762-b8fa-6cd9008a790b.json
 Almeda560_Davis923_c762bbaa-b207-4b4d-8123-e5bc93e3b99b.json
 Ana_María762_Gamez720_de3666a6-b4e3-4ec8-9ae7-bde168e03465.json
 Anne636_Collins926_42c179ad-99bd-4ca4-861f-7151d774a243.json
 Annelle169_King743_ddd8d7a4-5992-4572-a231-80bc305298b1.json
 Annemarie794_Kihn564_907083ad-22c9-4067-8321-a6622ce9792c.json
 Aracely711_Carter549_31665402-bfc5-45e1-b12a-fdc42836f50a.json
 Arden380_Reichel38_9c6e8766-9da5-428e-be61-e81a78a920c2.json
 Arlean815_MacGyver246_f9e2ec3e-731f-4341-8a6

In [ ]:
import json
from collections import defaultdict
import pandas as pd

# Load the JSON file to extract information
with open("/content/drive/MyDrive/synthea_dataset/Allegra202_Heller342_cc4c634f-c2ff-4963-b75a-fa2e33dc208f.json") as f:
    data = json.load(f)

patient={}

# Extract the entries
entries = data.get('entry', [])

# Group by resourceType
resource_dict = {}
for entry in entries:
    resource = entry.get('resource', {})
    resource_type = resource.get('resourceType', 'Unknown')
    if resource_type not in resource_dict:
        resource_dict[resource_type] = []
    resource_dict[resource_type].append(resource)


In [ ]:
def flatten_dataframe(df):
    for column in df.columns:
        if df[column].apply(lambda x: isinstance(x, (list, dict))).any():
            # Normalize the nested column
            normalized_df = pd.json_normalize(df[column].explode().dropna().tolist(), sep='_')
            normalized_df.columns = [f"{column}_{sub_col}" for sub_col in normalized_df.columns]
            # Drop the original nested column and join the normalized columns
            df = df.drop(columns=[column]).join(normalized_df)
    return df


In [ ]:
print(resource_dict.keys())

dict_keys(['Patient', 'Location', 'Organization', 'Practitioner', 'PractitionerRole', 'Encounter', 'Observation', 'DiagnosticReport', 'DocumentReference', 'Procedure', 'Condition', 'MedicationRequest', 'CareTeam', 'CarePlan', 'Immunization', 'ImagingStudy', 'Device', 'MedicationStatement'])


In [ ]:
for resource_type, resources in resource_dict.items():
    print(f"ResourceType: {resource_type}")
    # Normalize the entire resource dictionary and separate nested tags
    df = pd.json_normalize(resources, sep='_')
    #print(df.columns)

    # Flatten the nested columns
    df = flatten_dataframe(df)

    # Filter out inactive patient information
    if 'status' in df.columns:
      print(df['status'].unique())
        #df = df[(df['status'] == 'active') | (df['status'] == 'current') | (df['status'] == 'final') | (df['status'] == 'finished') | (df['status'] == 'completed')]

    if 'id' in df.columns:
        df = df.drop(columns=['id'])

    columns_to_drop = [col for col in df.columns if any(substring in col for substring in ['reference', 'system', 'telecom', 'identifier', 'Identifier', 'uid', 'Address', 'address'])]
    df = df.drop(columns=columns_to_drop)

    df = flatten_dataframe(df) #handle second layer of nesting

    print(df.columns)
    #print(df.head(2))

    patient[resource_type] = df
    print('\n')

ResourceType: Patient
Index(['resourceType', 'gender', 'birthDate', 'multipleBirthBoolean',
       'text_status', 'text_div', 'maritalStatus_text', 'extension_url',
       'extension_valueString', 'extension_valueCode',
       'extension_valueDecimal', 'name_use', 'name_family',
       'communication_language_text', 'maritalStatus_coding_code',
       'maritalStatus_coding_display', 'extension_extension_url',
       'extension_extension_valueCoding_system',
       'extension_extension_valueCoding_code',
       'extension_extension_valueCoding_display',
       'extension_extension_valueString',
       'communication_language_coding_system',
       'communication_language_coding_code',
       'communication_language_coding_display'],
      dtype='object')


ResourceType: Location
['active']
Index(['resourceType', 'status', 'name', 'position_longitude',
       'position_latitude', 'managingOrganization_display'],
      dtype='object')


ResourceType: Organization
Index(['resourceType', 'a

In [ ]:
for resource_type, resources in resource_dict.items():
  print(patient[resource_type].head(2))

  resourceType  gender   birthDate  multipleBirthBoolean text_status  \
0      Patient  female  1928-04-23                 False   generated   

                                            text_div maritalStatus_text  \
0  <div xmlns="http://www.w3.org/1999/xhtml">Gene...                  M   

                                       extension_url extension_valueString  \
0  http://hl7.org/fhir/us/core/StructureDefinitio...                   NaN   

  extension_valueCode  ...  maritalStatus_coding_code  \
0                 NaN  ...                          M   

  maritalStatus_coding_display extension_extension_url  \
0                            M             ombCategory   

  extension_extension_valueCoding_system extension_extension_valueCoding_code  \
0        urn:oid:2.16.840.1.113883.6.238                               2106-3   

  extension_extension_valueCoding_display extension_extension_valueString  \
0                                   White                             NaN  

In [ ]:
import requests
import pandas as pd

# Initial URL for the first API call
base_url = "https://clinicaltrials.gov/api/v2/studies"
params = {
    "query.titles": "Breast Cancer",
    "pageSize": 100
}

# Initialize an empty list to store the data
data_list = []

# Loop until there is no nextPageToken
while True:
    # Print the current URL (for debugging purposes)
    print("Fetching data from:", base_url + '?' + '&'.join([f"{k}={v}" for k, v in params.items()]))

    # Send a GET request to the API
    response = requests.get(base_url, params=params)

    # Check if the request was successful
    if response.status_code == 200:
        data = response.json()  # Parse JSON response
        studies = data.get('studies', [])  # Extract the list of studies

        # Loop through each study and extract specific information
        for study in studies:
            # Safely access nested keys
            nctId = study['protocolSection']['identificationModule'].get('nctId', 'Unknown')
            overallStatus = study['protocolSection']['statusModule'].get('overallStatus', 'Unknown')
            startDate = study['protocolSection']['statusModule'].get('startDateStruct', {}).get('date', 'Unknown Date')
            conditions = ', '.join(study['protocolSection']['conditionsModule'].get('conditions', ['No conditions listed']))
            acronym = study['protocolSection']['identificationModule'].get('acronym', 'Unknown')

            # Extract interventions safely
            interventions_list = study['protocolSection'].get('armsInterventionsModule', {}).get('interventions', [])
            interventions = ', '.join([intervention.get('name', 'No intervention name listed') for intervention in interventions_list]) if interventions_list else "No interventions listed"

            # Extract locations safely
            locations_list = study['protocolSection'].get('contactsLocationsModule', {}).get('locations', [])
            locations = ', '.join([f"{location.get('city', 'No City')} - {location.get('country', 'No Country')}" for location in locations_list]) if locations_list else "No locations listed"

            # Extract dates and phases
            primaryCompletionDate = study['protocolSection']['statusModule'].get('primaryCompletionDateStruct', {}).get('date', 'Unknown Date')
            studyFirstPostDate = study['protocolSection']['statusModule'].get('studyFirstPostDateStruct', {}).get('date', 'Unknown Date')
            lastUpdatePostDate = study['protocolSection']['statusModule'].get('lastUpdatePostDateStruct', {}).get('date', 'Unknown Date')
            studyType = study['protocolSection']['designModule'].get('studyType', 'Unknown')
            phases = ', '.join(study['protocolSection']['designModule'].get('phases', ['Not Available']))

            # Extract eligibility criteria
            #eligibilities = ', '.join(study['protocolSection']['eligibilityModule'].get('eligibilityCriteria', ['No criteria listed']))

            eligibility = study['protocolSection']['eligibilityModule'].get('eligibilityCriteria', 'Unknown')

            #eligibilities_list = study['protocolSection'].get('eligibilityModule', {}).get('eligibilityCriteria', [])
            #eligibilities = ', '.join([eligiblity.get('name', 'No criteria listed') for eligiblity in eligibilities_list]) if eligibilities_list else "No eligibility listed"

            # Append the data to the list as a dictionary
            data_list.append({
                "NCT ID": nctId,
                "Acronym": acronym,
                "Overall Status": overallStatus,
                "Start Date": startDate,
                "Conditions": conditions,
                "Interventions": interventions,
                "Locations": locations,
                "Primary Completion Date": primaryCompletionDate,
                "Study First Post Date": studyFirstPostDate,
                "Last Update Post Date": lastUpdatePostDate,
                "Study Type": studyType,
                "Phases": phases,
                "Eligibility": eligibility
            })

        # Check for nextPageToken and update the params or break the loop
        nextPageToken = data.get('nextPageToken')
        if nextPageToken:
            params['pageToken'] = nextPageToken  # Set the pageToken for the next request
        else:
            break  # Exit the loop if no nextPageToken is present
    else:
        print("Failed to fetch data. Status code:", response.status_code)
        break

# Create a DataFrame from the list of dictionaries
clinical_trials_data = pd.DataFrame(data_list)

# Print the DataFrame
#print(clinical_trials_data)

# Optionally, save the DataFrame to a CSV file
clinical_trials_data.to_csv("clinical_trials_data.csv", index=False)

Fetching data from: https://clinicaltrials.gov/api/v2/studies?query.titles=Breast Cancer&pageSize=100
Fetching data from: https://clinicaltrials.gov/api/v2/studies?query.titles=Breast Cancer&pageSize=100&pageToken=NF0g5JCFl_Ms
Fetching data from: https://clinicaltrials.gov/api/v2/studies?query.titles=Breast Cancer&pageSize=100&pageToken=NF0g5JWCk_go
Fetching data from: https://clinicaltrials.gov/api/v2/studies?query.titles=Breast Cancer&pageSize=100&pageToken=NF0g5JqClvgr
Fetching data from: https://clinicaltrials.gov/api/v2/studies?query.titles=Breast Cancer&pageSize=100&pageToken=NF0g5JKFlPIhww
Fetching data from: https://clinicaltrials.gov/api/v2/studies?query.titles=Breast Cancer&pageSize=100&pageToken=NF0g5JKClfIhwA
Fetching data from: https://clinicaltrials.gov/api/v2/studies?query.titles=Breast Cancer&pageSize=100&pageToken=NF0g5JKPlPkqyA
Fetching data from: https://clinicaltrials.gov/api/v2/studies?query.titles=Breast Cancer&pageSize=100&pageToken=NF0g5JGFlvgrxA
Fetching data f

In [ ]:
clinical_trials_data = clinical_trials_data[clinical_trials_data['Overall Status'] == 'RECRUITING']

In [ ]:
pip install openai --upgrade

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 337.0/337.0 kB 15.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 4.3 MB/s eta 0:00:00


In [ ]:
!pip install langchain_openai --upgrade

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.2/48.2 kB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 378.9/378.9 kB 16.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 18.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 275.8/275.8 kB 10.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 141.1/141.1 kB 9.1 MB/s eta 0:00:00
  Attempting uninstall: tenacity
    Found existing installation: tenacity 9.0.0
    Uninstalling tenacity-9.0.0:
      Successfully uninstalled tenacity-9.0.0


In [ ]:
import os
from langchain_openai import ChatOpenAI

# os.environ["OPENAI_API_KEY"] = 'sk-proj-XIZYSWf9cabLNiQ4h9HvT3BlbkFJZXRUtBkclFgKigJ7WHuH' # go to platform.openai.com to find your key!
os.environ["OPENAI_API_KEY"] = 'sk-proj-bbcCqgJsIPOhjmiRfNdyT3BlbkFJuDf9IePweh97i2lYZkTl' # API Key from Jieming for MVP
# model
model = "gpt-4-turbo" # most powerful gpt model
llm = ChatOpenAI(temperature=0.1, model=model)

In [ ]:
from openai import OpenAI

client = OpenAI(
  api_key=os.environ["OPENAI_API_KEY"],  # this is also the default, it can be omitted
)

### **Part 1:** Querying the LLM with the full health record and the full clinical trials dataframe.

In [ ]:
from datetime import date
today_date = date.today()

query = f"""You are a doctor seeing a patient diagnosed with a type of breast cancer with the following medical history. {patient}. Extract information about the patient's cancer prognosis and treatment. The patient's age can be inferred from today's data, {today_date} and the patient’s date of birth. Use this information and the clinical trial inclusion/exclusion criteria to identify up to 4 clinical trials that the patient is eligible for. The patient must meet all of the criteria listed for the clinical trial. Identify and report the NCT ID number and the clinical trial eligibility criteria. List four reasons why the patient is eligible for each trial. If the patient does not match the criteria for any of the trials, explain why.{clinical_trials_data}'
"""

#print(query)

response = llm.invoke(query)

In [ ]:
print(response.content)

Based on the provided data, the patient is a female born on April 23, 1928. As of today's date, August 1, 2024, the patient is 96 years old. The patient has been diagnosed with breast cancer, as indicated by the presence of conditions and treatments related to breast cancer in her medical history.

To find suitable clinical trials for the patient, we need to consider her age, the specific type of breast cancer, and any other relevant medical conditions or treatments she has undergone. The clinical trials must have eligibility criteria that the patient meets.

### Analysis of Clinical Trials:
Given the vast number of clinical trials, I will focus on those specifically targeting breast cancer and check if the patient meets the age and other specific criteria mentioned.

#### Selected Clinical Trials:
1. **NCT05939830**
   - **Eligibility Criteria**:
     - Female aged between 18 and 80 years.
     - Diagnosed with breast cancer.
     - No prior chemotherapy or radiotherapy.
     - Adequa

In [ ]:
print(response.response_metadata)

{'token_usage': {'completion_tokens': 741, 'prompt_tokens': 25681, 'total_tokens': 26422}, 'model_name': 'gpt-4-turbo-2024-04-09', 'system_fingerprint': 'fp_0993c4a4c0', 'finish_reason': 'stop', 'logprobs': None}


### **Part 2:** Prompt the LLM to write a patient summary, given a full health record.

#### Summary 1

In [ ]:
query = f"""You are a doctor seeing a patient diagnosed with a type of breast cancer with the following health record in the mCODE format. Write a summary for this patient's cancer prognosis. {patient}'
"""

response = llm.invoke(query)

print(response.content)

The patient, Mrs. Allegra202 Heller342, is a female born on April 23, 1928, and has been diagnosed with breast cancer. She is married and identifies as White. Her primary language is English. The patient has been under the care of healthcare providers at Morton Hospital and PCP187044, with Dr. Onita767 Osinski784 and Dr. Sharilyn202 Simonis280 involved in her care.

Her medical history is extensive, with multiple encounters and procedures documented. Notably, she has had several conditions over the years, including otitis media, acute bronchitis, viral sinusitis, normal pregnancies, essential hypertension, and more recently, coronary heart disease, myocardial infarction, and osteoporosis. Her condition of breast cancer, identified as "Malignant neoplasm of breast (disorder)," is currently active and has been managed as part of her ongoing care.

Mrs. Heller342 has been prescribed various medications over the years, including Aspirin, Penicillin V Potassium, Hydrochlorothiazide, Nitrogl

#### Summary 2

In [ ]:
query = f"""You are a doctor seeing a patient diagnosed with a type of breast cancer with the following health record in the mCODE format. Write a detailed summary for this patient's cancer diagnosis and prognosis. Report the clinical stage type of the cancer diagnosis and other descriptive information about the cancer. Report the gender and the patient's age as of today's date, {today_date}. Report the patient's comorbidities and treatments. {patient}'
"""

response = llm.invoke(query)

print(response.content)

### Patient Summary

**Patient Details:**
- **Name:** Mrs. Allegra202 Heller342
- **Gender:** Female
- **Date of Birth:** April 23, 1928
- **Age:** 96 years old (as of July 30, 2024)
- **Marital Status:** Married
- **Race:** White
- **Primary Language:** English

**Medical History:**
- **Diagnosis:** Malignant neoplasm of breast (disorder)
- **Clinical Stage:** Stage 2A (qualifier value)
- **Comorbidities:**
  - Essential hypertension (disorder)
  - Hyperlipidemia
  - History of myocardial infarction (situation)
  - Osteoporosis (disorder)
- **Treatments:**
  - Medications:
    - Aspirin 81 MG Oral Tablet
    - Simvastatin 20 MG Oral Tablet
    - Hydrochlorothiazide 25 MG Oral Tablet
    - Nitroglycerin 400 MCG/ACTUAT Mucosal Spray
    - AmLODIPine besylate 5 MG Oral Tablet
  - Procedures:
    - Mammography
    - Manual pelvic examination
    - Cytopathology procedure, preparation of smear
  - Immunizations:
    - Influenza, seasonal, injectable
    - Td (adult), 5 Lf tetanus toxoid
- 

In [ ]:
patient_summary = response.content

In [ ]:
print(patient_summary)

### Patient Summary

**Patient Details:**
- **Name:** Mrs. Allegra202 Heller342
- **Gender:** Female
- **Date of Birth:** April 23, 1928
- **Age:** 96 years old (as of July 30, 2024)
- **Marital Status:** Married
- **Race:** White
- **Primary Language:** English

**Medical History:**
- **Diagnosis:** Malignant neoplasm of breast (disorder)
- **Clinical Stage:** Stage 2A (qualifier value)
- **Comorbidities:**
  - Essential hypertension (disorder)
  - Hyperlipidemia
  - History of myocardial infarction (situation)
  - Osteoporosis (disorder)
- **Treatments:**
  - Medications:
    - Aspirin 81 MG Oral Tablet
    - Simvastatin 20 MG Oral Tablet
    - Hydrochlorothiazide 25 MG Oral Tablet
    - Nitroglycerin 400 MCG/ACTUAT Mucosal Spray
    - AmLODIPine besylate 5 MG Oral Tablet
  - Procedures:
    - Mammography
    - Manual pelvic examination
    - Cytopathology procedure, preparation of smear
  - Immunizations:
    - Influenza, seasonal, injectable
    - Td (adult), 5 Lf tetanus toxoid
- 

### **Part 3:** Querying the LLM with the patient summary and the full clinical trials dataframe.

In [ ]:
query = f"""You are a doctor seeing a patient diagnosed with a type of breast cancer with the following medical history. {patient_summary}. Use this information and the clinical trial inclusion/exclusion criteria to identify up to 4 clinical trials that the patient is eligible for. The patient must meet all of the criteria listed for the clinical trial. Identify and report the NCT ID number and the clinical trial eligibility (inclusion and exclusion) criteria. For each eligibility criterion for the trial, explain why the patient is eligible. If the patient does not match the criteria for any of the trials, explain why. {clinical_trials_data}.’
"""

response = llm.invoke(query)

print(response.content)

content='Based on the patient\'s profile and the clinical trial information provided, we need to assess whether Mrs. Allegra Heller meets the eligibility criteria for any of the listed clinical trials. Let\'s evaluate her eligibility for each trial based on the inclusion criteria provided:\n\n### Trial NCT06155331\n- **Eligibility Criteria:**\n  - Age ≥ 18 years old.\n  - Diagnosed with Stage 2 or 3 breast cancer.\n  - Able to provide informed consent.\n  - No prior chemotherapy for this cancer.\n\n**Reasons for Ineligibility:**\n1. **Age:** Mrs. Heller is 96 years old, which might be beyond the typical upper age limit for clinical trials, although not specified here.\n2. **Prior Treatments:** The patient\'s medical history does not specify prior chemotherapy, but given her stage and ongoing management, it\'s possible she might have received it.\n3. **General Health:** Given her age and multiple comorbidities, she might not be considered a suitable candidate for a trial involving addit

### **Part 4:** Querying the LLM with the patient summary and the clinical trials data using a RAG-enabled approach

In [ ]:
clinical_trials_data['text'] = 'The trial with NCT ID number ' + clinical_trials_data['NCT ID'] + ' has the following eligbility criteria, for inclusion and exclusion ' + clinical_trials_data['Eligibility']
clinical_trials_data.head()['text']

,text
14,"The trial with NCT ID number NCT06155331 has the following eligbility criteria, for inclusion and exclusion Inclusion Criteria:\n\n* Age ≥ 18 years old.\n* Patients with biopsy confirmed diagnosis breast cancer and with stage II and stage III breast cancer according to the American Joint Committee on Cancer (TNM staging system of breast cancer).\n* Patients with performance status \<2 according to Eastern Cooperative Oncology Group (ECOG) score.\n* Adequate baseline hematologic values (absolute neutrophilic count ≥ 1.5 × 109/L, platelet count ≥ 100 × 109/L and hemoglobin level ≥ 10 g/dl).\n* Patients with adequate liver function (serum bilirubin \< 1.2 mg/dl) and adequate renal function (serum creatinine \< 1.5 mg/d).\n\nExclusion Criteria:\n\n* Patients with prior exposure to anthracyclines in the last 6 months.\n* Patients with evidence of metastasis at the initial assessment.\n* Concomitant use of antioxidant vitamins (vitamin A, C, E).\n* Presence of clinical evidence for severe cardiac illness (angina pectoris, uncontrolled hypertension, arrhythmias and left ventricular ejection fraction \<50%).\n* Patients with inflammatory diseases (ulcerative colitis, rheumatoid arthritis).\n* Patients with conditions associated with oxidative stress (smoking, tuberculosis, comorbid obesity).\n* Patients who are candidates for monoclonal antibodies such as Trastuzumab and other targeted therapy (HER2 positive patients).\n* Patients with active liver disease (cirrhosis, fatty liver, hepatitis C, etc..).\n* Patients with myopathy.\n* Patients with renal impairment, including those with end-stage renal disease and those receiving dialysis.\n* Pregnant and breast feeding women.\n* Known allergy to the fenofibrates.\n* Concurrent use of statin, colchicine, ciprofibrate, idelalisib, ivacaftor, aspirin low strength, clopidogrel, warfarin, enzyme inducers (phenytoin, phenobarbitone, carbamazepine,...), enzyme inhibitors (allopurinol, MAOI, SSRI,...), drugs with high plasma protein binding capacity (sulfonamides, valproate, oral hypoglycemic, warfarin,...) in order to avoid potential pharmacodynamics and pharmacokinetic drug interactions."
34,"The trial with NCT ID number NCT04603820 has the following eligbility criteria, for inclusion and exclusion Inclusion Criteria:\n\n1. The patient should have signed and dated the informed consent form (ICF). The enrollment of patients who have died is allowed.\n2. Women aged ≥ 18 years.\n3. Patients in one of the following situations:\n\n * Patients with breast cancer diagnosis during pregnancy, breastfeeding or within the year after delivery.\n * Patients with breast cancer who become pregnant after treatment.\n * Patients with breast cancer who were subjected to any fertility preservation method prior to the start of breast cancer treatment.\n4. The patients referred to in the previous section and the patients who meet these characteristics prospectively could be enrolled retrospectively upon registry opening.\n5. All cases diagnosed at the same site may be included. In order to prevent duplications, in case the patient followed her treatment and follow-up at another site, she will be enrolled as per the site where the diagnosis was made, requesting information of the treatment and progression, when possible.\n6. Availability of clinical, epidemiological and progress data.\n\nExclusion Criteria:\n\nPatients who do not wish to participate in the study for any reason could not be included in the study."
35,"The trial with NCT ID number NCT04074720 has the following eligbility criteria, for inclusion and exclusion Inclusion Criteria:\n\n* Adults \>18 years old at time of consent\n* Subject must be capable to giving informed consent or have an acceptable surrogate capable of giving consent on the subject behalf.\n* Patients with carcinoma in situ or invasive breast cancer\n* Patient must be undergoing one of the following:\n\n * definitive surgical tumor resection for breast cancer OR\n * placemen

In [ ]:
import openai
import tiktoken
from scipy import spatial
import pandas as pd
import ast
import openai

# Function to get text embeddings
def text_embedding(text):
    response = client.embeddings.create(model="text-embedding-ada-002", input=text)
    return response.data[0].embedding

# Apply the text_embedding function to the 'text' column
clinical_trials_data['embedding'] = clinical_trials_data['text'].apply(lambda x: text_embedding(x))

#clinical_trials_data = clinical_trials_data.assign(embedding = (clinical_trials_data["text"].apply(lambda x : text_embedding(x))))
print(clinical_trials_data.head())

         NCT ID   Acronym Overall Status  Start Date  \
14  NCT06155331   Unknown     RECRUITING     2023-12   
34  NCT04603820  EMBARCAM     RECRUITING  2019-11-18   
35  NCT04074720   Unknown     RECRUITING  2018-05-10   
38  NCT06087120   Unknown     RECRUITING  2023-09-16   
39  NCT01785420   Unknown     RECRUITING     2013-05   

                                           Conditions  \
14                        Breast Cancer Stage 2 and 3   
34                        Breast Cancer and Pregnancy   
35  Breast Cancer, Invasive Breast Cancer, Carcino...   
38  Breast Cancer Female, Stage II Breast Cancer, ...   
39  Carcinoma Breast Stage I, HER2 Positive Breast...   

                                        Interventions  \
14  Fenofibrate, Placebo, Doxorubicin, Cyclophosph...   
34                            No interventions listed   
35  Tissue Sample collection, Blood Sample Collect...   
38                            No interventions listed   
39                               Tr

In [ ]:
def strings_ranked_by_relatedness(
    query: str,
    df: pd.DataFrame,
    relatedness_fn = lambda x, y: 1 - spatial.distance.cosine(x, y),
    top_n: int = 1000
):

    EMBEDDING_MODEL = "text-embedding-ada-002"
    query_embedding_response = openai.embeddings.create(
        model = EMBEDDING_MODEL,
        input = query,
    )
    query_embedding = query_embedding_response.data[0].embedding
    strings_and_relatednesses = [
        (row["text"], relatedness_fn(query_embedding, row["embedding"]))
        for i, row in df.iterrows()
    ]
    strings_and_relatednesses.sort(key = lambda x: x[1], reverse = True)
    strings, relatednesses = zip(*strings_and_relatednesses)
    return strings[:top_n], relatednesses[:top_n]

def num_tokens(text: str) -> int:
    encoding = tiktoken.encoding_for_model("gpt-4o")
    return len(encoding.encode(text))

def query_message(
    query: str,
    df: pd.DataFrame,
    model: str,
    token_budget: int
) :
    strings, relatednesses = strings_ranked_by_relatedness(query, df)
    introduction = 'Use the below content to match patients to clinical trials. Explain why the patient does not match any of the eligibility criteria."'
    question = f"\n\nQuestion: {query}"
    message = introduction
    for string in strings:
        next_row = f'\n\nClinical trial section:\n"""\n{string}\n"""'
        if (
            num_tokens(message + next_row + question)
            > token_budget
        ):
            break
        else:
            message += next_row
    return message + question

def ask(
    query: str,
    df: pd.DataFrame = clinical_trials_data,
    model: str = "gpt-4o",
    token_budget: int = 25000,
    print_message: bool = False,
) :
    message = query_message(query, df, model=model, token_budget = token_budget)
    if print_message:
        print(message)
    messages = [
        {"role": "system", "content": "You match patients to clinical trials that they are eligible for."},
        {"role": "user", "content": message},
    ]
    response = openai.chat.completions.create(
        model = model,
        messages = messages,
        temperature = 0
    )
    #response_message = response["choices"][0]["message"]["content"]
    response_message = response.choices[0].message.content
    return response_message

relatedness = 0.817


'The trial with NCT ID number NCT05044689 has the following eligbility criteria, for inclusion and exclusion Case Group - Patients must have histologically confirmed diagnosis of colorectal or breast cancer.\n\nControl Group - Patients must not have a diagnosis of cancer.'

relatedness = 0.816


'The trial with NCT ID number NCT05406531 has the following eligbility criteria, for inclusion and exclusion Inclusion Criteria:\n\n* Women over the age of 18\n* Breast cancer patients at the end of adjuvant treatment\n\nExclusion Criteria:\n\n* Patients without a mobil device ("smart" mobile phone)\n* Patients without internet access\n* Patients with diagnosed: psychotic disorder; severe depressive episode; panic disorder; bipolar disorder; personality disorder associated with uncontrollable mood swings; PTSD; suicidal attempts and active suicidal ideation; disability pension due to non-specific mental or somatic problems; hospitalization due to a psychiatric disorder in the past year'

relatedness = 0.816


'The trial with NCT ID number NCT05704842 has the following eligbility criteria, for inclusion and exclusion Inclusion Criteria:\n\n* Women with curative-intent breast cancer who plan to undergo at least 4 cycles of chemotherapy.\n* Age \\> 18 years\n* ECOG performance score \\< 3\n* English-speaking\n* with sufficient vision/hearing or family support\n* Coronary artery disease, if cleared by cardiologist\n* Subject must have smart phone, computer or tablet.\n* Willingness to be randomized\n\nExclusion Criteria:\n\n* Medical or psychiatric conditions (beyond those related to breast cancer and its treatment) that would impair our ability to test study hypotheses (psychotic disorders, dementia, inability to give informed consent or follow instructions).\n* Patients with overt evidence of a psychiatric disorder.\n* Coronary artery disease, not cleared by cardiologist.\n* Contraindication to exercise.\n* Chronic fatigue syndrome.'

In [ ]:
query = f"""You are a doctor seeing a patient diagnosed with a type of breast cancer with the following medical history. {patient_summary}. Use this information and the clinical trial inclusion/exclusion criteria to identify up to 4 clinical trials that the patient is eligible for. The patient must meet all of the criteria listed for the clinical trial. Identify and report the NCT ID number and the clinical trial eligibility (inclusion and exclusion) criteria. For each eligibility criterion for the trial, explain why the patient is eligible. If the patient does not match the criteria for any of the trials, explain why."""

In [ ]:
resp_0 = ask(query)

In [ ]:
print(resp_0)

Based on the provided patient summary and the clinical trial eligibility criteria, Mrs. Allegra202 Heller342 does not match the criteria for any of the clinical trials listed. Here is a detailed explanation of why she does not meet the eligibility criteria for each trial:

1. **NCT06035679**
   - **Inclusion Criteria:**
     - Age 18-75 years: Mrs. Heller is 96 years old.
     - Her2-positive breast cancer: Not specified in the patient summary.
     - ECOG PS: 0-1: Not specified in the patient summary.
     - Plan for surgical removal: Not specified in the patient summary.
     - Normal function of major organs: Not specified in the patient summary.
     - Effective contraception: Not applicable due to age.
   - **Exclusion Criteria:**
     - History of myocardial infarction: Mrs. Heller has a history of myocardial infarction.
     - Age: Mrs. Heller is 96 years old, which exceeds the age limit.

2. **NCT05867667**
   - **Inclusion Criteria:**
     - Age 18 years or older: Mrs. Heller 

### **Part 5:** Exploration of ranking Inclusion and Exclusion Criteria separately

In [ ]:
new_data = clinical_trials_data.copy()
new_data = new_data.drop(['embedding', 'text'], axis=1)


Index(['NCT ID', 'Acronym', 'Overall Status', 'Start Date', 'Conditions',
       'Interventions', 'Locations', 'Primary Completion Date',
       'Study First Post Date', 'Last Update Post Date', 'Study Type',
       'Phases', 'Eligibility'],
      dtype='object')


In [ ]:
print(str(new_data['Eligibility'].iloc[0]))

Inclusion Criteria:

* Age ≥ 18 years old.
* Patients with biopsy confirmed diagnosis breast cancer and with stage II and stage III breast cancer according to the American Joint Committee on Cancer (TNM staging system of breast cancer).
* Patients with performance status \<2 according to Eastern Cooperative Oncology Group (ECOG) score.
* Adequate baseline hematologic values (absolute neutrophilic count ≥ 1.5 × 109/L, platelet count ≥ 100 × 109/L and hemoglobin level ≥ 10 g/dl).
* Patients with adequate liver function (serum bilirubin \< 1.2 mg/dl) and adequate renal function (serum creatinine \< 1.5 mg/d).

Exclusion Criteria:

* Patients with prior exposure to anthracyclines in the last 6 months.
* Patients with evidence of metastasis at the initial assessment.
* Concomitant use of antioxidant vitamins (vitamin A, C, E).
* Presence of clinical evidence for severe cardiac illness (angina pectoris, uncontrolled hypertension, arrhythmias and left ventricular ejection fraction \<50%).
* P

In [ ]:
def split_eligibility(row):
    parts = row['Eligibility'].split('Exclusion Criteria:')
    if len(parts) == 2:
        inclusion_criteria = parts[0].strip()
        exclusion_criteria = parts[1].strip()
    else:
        inclusion_criteria = row['Eligibility'].strip()
        exclusion_criteria = ''
    return inclusion_criteria, exclusion_criteria

# Apply the function to each row
new_data[['Inclusion_Criteria', 'Exclusion_Criteria']] = new_data.apply(split_eligibility, axis=1, result_type='expand')

# Create the new columns with the formatted strings
new_data['text_inclusion'] = 'The trial with NCT ID number ' + new_data['NCT ID'] + ' has the following eligibility inclusion criteria: ' + new_data['Inclusion_Criteria']
new_data['text_exclusion'] = 'The trial with NCT ID number ' + new_data['NCT ID'] + ' has the following eligibility exclusion criteria: Exclusion Criteria: ' + new_data['Exclusion_Criteria']

# Display the head of the new columns
print(new_data.head()[['text_inclusion', 'text_exclusion']])

                                       text_inclusion  \
14  The trial with NCT ID number NCT06155331 has t...   
34  The trial with NCT ID number NCT04603820 has t...   
35  The trial with NCT ID number NCT04074720 has t...   
38  The trial with NCT ID number NCT06087120 has t...   
39  The trial with NCT ID number NCT01785420 has t...   

                                       text_exclusion  
14  The trial with NCT ID number NCT06155331 has t...  
34  The trial with NCT ID number NCT04603820 has t...  
35  The trial with NCT ID number NCT04074720 has t...  
38  The trial with NCT ID number NCT06087120 has t...  
39  The trial with NCT ID number NCT01785420 has t...  


In [ ]:
import openai
import tiktoken
from scipy import spatial
import pandas as pd
import ast
import openai

# Function to get text embeddings
def text_embedding(text):
    response = client.embeddings.create(model="text-embedding-ada-002", input=text)
    return response.data[0].embedding

# Apply the text_embedding function to the 'text' column
new_data['embedding_inclusion'] = new_data['text_inclusion'].apply(lambda x: text_embedding(x))
new_data['embedding_exclusion'] = new_data['text_exclusion'].apply(lambda x: text_embedding(x))

print(new_data.head())

         NCT ID   Acronym Overall Status  Start Date  \
14  NCT06155331   Unknown     RECRUITING     2023-12   
34  NCT04603820  EMBARCAM     RECRUITING  2019-11-18   
35  NCT04074720   Unknown     RECRUITING  2018-05-10   
38  NCT06087120   Unknown     RECRUITING  2023-09-16   
39  NCT01785420   Unknown     RECRUITING     2013-05   

                                           Conditions  \
14                        Breast Cancer Stage 2 and 3   
34                        Breast Cancer and Pregnancy   
35  Breast Cancer, Invasive Breast Cancer, Carcino...   
38  Breast Cancer Female, Stage II Breast Cancer, ...   
39  Carcinoma Breast Stage I, HER2 Positive Breast...   

                                        Interventions  \
14  Fenofibrate, Placebo, Doxorubicin, Cyclophosph...   
34                            No interventions listed   
35  Tissue Sample collection, Blood Sample Collect...   
38                            No interventions listed   
39                               Tr

In [ ]:
new_data['text'] = 'The trial with NCT ID number ' + new_data['NCT ID'] + ' has the following eligbility criteria, for inclusion and exclusion ' + new_data['Eligibility']

In [ ]:
# Function to compute the combined score
def combined_score(query_embedding, row, relatedness_fn, unrelatedness_fn):
    relatedness = relatedness_fn(query_embedding, row["embedding_inclusion"])
    unrelatedness = unrelatedness_fn(query_embedding, row["embedding_exclusion"])
    return relatedness - unrelatedness  # Higher relatedness and lower unrelatedness

# Function to rank strings by combined score
def strings_ranked_by_combined_score(
    query: str,
    df: pd.DataFrame,
    relatedness_fn = lambda x, y: 1 - spatial.distance.cosine(x, y),
    unrelatedness_fn = lambda x, y: spatial.distance.cosine(x, y),
    top_n: int = 1000
):
    EMBEDDING_MODEL = "text-embedding-ada-002"
    query_embedding_response = openai.embeddings.create(
        model = EMBEDDING_MODEL,
        input = query,
    )
    query_embedding = query_embedding_response.data[0].embedding
    strings_and_scores = [
        (row["text"], combined_score(query_embedding, row, relatedness_fn, unrelatedness_fn))
        for i, row in df.iterrows()
    ]
    strings_and_scores.sort(key=lambda x: x[1], reverse=True)
    strings, scores = zip(*strings_and_scores)
    return strings[:top_n], scores[:top_n]

# Function to calculate number of tokens
def num_tokens(text: str) -> int:
    encoding = tiktoken.encoding_for_model("gpt-4o")
    return len(encoding.encode(text))

# Function to generate query message
def query_message(
    query: str,
    df: pd.DataFrame,
    model: str,
    token_budget: int
):
    strings, scores = strings_ranked_by_combined_score(query, df)
    introduction = 'Use the below content to match patients to clinical trials. Explain why the patient does not match any of the eligibility criteria."'
    question = f"\n\nQuestion: {query}"
    message = introduction
    for string in strings:
        next_row = f'\n\nClinical trial section:\n"""\n{string}\n"""'
        if num_tokens(message + next_row + question) > token_budget:
            break
        else:
            message += next_row
    return message + question

# Function to ask the model
def ask(
    query: str,
    df: pd.DataFrame = new_data,
    model: str = "gpt-4o",
    token_budget: int = 25000,
    print_message: bool = False,
):
    message = query_message(query, df, model=model, token_budget=token_budget)
    if print_message:
        print(message)
    messages = [
        {"role": "system", "content": "You match patients to clinical trials that they are eligible for."},
        {"role": "user", "content": message},
    ]
    response = openai.chat.completions.create(
        model=model,
        messages=messages,
        temperature=0
    )
    #response_message = response.choices[0].message.content
    return response_message


In [ ]:
query = f"""You are a doctor seeing a patient diagnosed with a type of breast cancer with the following medical history. {patient_summary}. Use this information and the clinical trial inclusion/exclusion criteria to identify up to 4 clinical trials that the patient is eligible for. The patient must meet all of the criteria listed for the clinical trial. Identify and report the NCT ID number and the clinical trial eligibility (inclusion and exclusion) criteria. For each eligibility criterion for the trial, explain why the patient is eligible. If the patient does not match the criteria for any of the trials, explain why."""

In [ ]:
resp_0 = ask(query)

In [ ]:
print(resp_0)

Based on the provided patient summary and the clinical trial eligibility criteria, Mrs. Allegra202 Heller342 does not match the criteria for any of the clinical trials listed. Here are the reasons why she does not meet the eligibility criteria for each trial:

### NCT04858529
**Inclusion Criteria:**
- Age > 18 years old: **Eligible**
- Histologically confirmed invasive breast carcinoma: **Eligible**
- Clinical stage II-IIIC at presentation: **Eligible**
- HER2-positive breast cancer: **Unknown (not specified in patient summary)**
- Known hormone receptor status (ER and PR): **Unknown (not specified in patient summary)**
- ECOG Performance Status 0 or 1: **Unknown (not specified in patient summary)**
- Completed all necessary baseline laboratory and radiologic examinations prior to randomization: **Unknown (not specified in patient summary)**
- Baseline LVEF ≥ 55%: **Unknown (not specified in patient summary)**
- Women who are not postmenopausal or surgically sterile must agree to use c

References:

OpenAI et. al. (2023). GPT-4 Technical Report. https://arxiv.org/abs/2303.08774

OpenAI. (2024). Hello GPT-4o. https://openai.com/index/hello-gpt-4o/
Luo et. al. (2022). BioGPT: Generative Pre-trained Transformer for Biomedical Text Generation and Mining. https://arxiv.org/abs/2210.10341
Clinicaltrials.gov.

NIH: User Guide for Clinicaltrials.gov Website: https://clinicaltrials.gov/submit-studies/prs-help/user-guide#intro

Jin, Q., Wang, Z., Floudas, C. S., Chen, F., Gong, C., Bracken-Clarke, D., ... & Lu, Z. (2023). Matching patients to clinical trials with large language models. ArXiv.arXiv:2307.15051v4. Accessed on August 2, 2024. Available from: https://www.ncbi.nlm.nih.gov/pmc/articles/PMC10418514/
Koopman, B., & Zuccon, G. (2016, July). A test collection for matching patients to clinical trials. In Proceedings of the 39th International ACM SIGIR conference on Research and Development in Information Retrieval (pp. 669-672).

Msv, J. (2023, July 21). Tutorial: Build a Q&A bot for Academy Awards based on ChatGPT. The New Stack. https://thenewstack.io/tutorial-build-a-qa-bot-for-academy-awards-based-on-chatgpt/
Stack OverFlow, Accessed on August 4th 2024. https://stackoverflow.com/questions/78415818/how-to-get-full-results-with-clinicaltrials-gov-api-in-python

Roberts, K., Demner-Fushman, D., Voorhees, E.M., Bedrick, S. & Hersh, W.R. Overview of the TREC 2021 Clinical Trials Track. in Proceedings of the Thirtieth Text REtrieval Conference (TREC 2021) (2021).

Walonoski, J., Kramer, M., Nichols, J., Quina, A., Moesel, C., Hall, D., ... & McLachlan, S. (2018). Synthea: An approach, method, and software mechanism for generating synthetic patients and the synthetic electronic health care record. Journal of the American Medical Informatics Association, 25(3), 230-238.

Ceylan, B., & Özerdoğan, N. (2015). Factors affecting age of onset of menopause and determination of quality of life in menopause. Turkish journal of obstetrics and gynecology, 12(1), 43–49. https://doi.org/10.4274/tjod.79836
Swanner, K. D., & Richmond, L. B. (2023). A 65-Year-Old Woman With No Menopause History: A Case Report. Cureus, 15(9), e44792. https://doi.org/10.7759/cureus.44792
